In [1]:
!pip install scikit-learn pandas datasets transformers torchvision evaluate accelerate

In [1]:
import torch
from utils import xception, TransferModel, create_path_label_dataset, get_transforms, evaluate_model, DeepFakeDetector

In [2]:
old_model = xception()
model = TransferModel(model=old_model,
                             num_out_classes=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [4]:
folder_true = '/home/jovyan/shared/S25/nochange/true'
folder_false = '/home/jovyan/shared/S25/nochange/false'

transforms = get_transforms(old_model)
dataset = create_path_label_dataset(folder_true, folder_false)
dataset = dataset.train_test_split(test_size=0.2)
dataset = dataset.with_transform(transforms)

labels = dataset['train'].features["label"].names

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/shared/S25/nochange/true'

In [3]:
import evaluate
import numpy as np
from transformers import TrainingArguments, Trainer, DefaultDataCollator

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="deepfake-unchanged",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DefaultDataCollator(),
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

NameError: name 'dataset' is not defined

In [7]:
# trainer.train()
labels

device(type='cpu')

In [8]:
from safetensors.torch import load

from utils import xception_from_tensors
new_model = xception_from_tensors("checkpoints/unchanged-model.safetensors")

In [9]:
detector = DeepFakeDetector("deepfake-1/checkpoint-153/model.safetensors")
detector.detect('/home/jovyan/shared/S25/nochange/false/000_003_frame247.png')

FileNotFoundError: [Errno 2] No such file or directory: 'deepfake-1/checkpoint-153/model.safetensors'

In [12]:
base_path = '/mnt/c/Users/28379/OneDrive/Desktop/AI/proj/Datasets'
folder_true = base_path + '/ValidationCeleb/Real1000'
folder_false = base_path + '/ValidationCeleb/Fake1000'

test_ds = create_path_label_dataset(folder_true, folder_false).with_transform(transforms)
evaluate_model(model, test_ds, compute_metrics)

/home/renhangx/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
